In [1]:
%cd ..

c:\Users\saleh\Documents\GitHub\livekit_my_project\backend


In [80]:
from abc import ABC
from pydantic import BaseModel
from typing import Optional, Any, List
import re
import pickle
from pydantic import (
    BaseModel, Field, EmailStr, create_model, computed_field
)
from typing import Literal

In [ ]:
from map.utils.create_schema import build_input_schema

In [26]:
class Test(BaseModel):
    x_path: str
    desc: str

In [ ]:
class Step(BaseModel, ABC):
    step_type: str
    x_path: str
    desc: str
    
    def run(self):
        raise NotImplementedError("Subclasses must implement this method")
    
class LeftClickStep(Step):
    desc: str = "Left click on the element"
    step_type: Literal["left_click"] = "left_click"
    
    def run(self):
        return f"Left clicking on element at {self.x_path}"

class InputStep(Step):
    desc: str = "Input text into the element"
    step_type: Literal["input"] = "input"

    # 
    input_schema_def: dict | None = None

    input_data: dict | None = None

    @property
    def input_schema(self) -> type[BaseModel] | None:
        if self.input_schema_def is None:
            return None
        return build_input_schema(self.input_schema_def)

    def validate_input(self, input_data: dict) -> Any:
        try:
            self.input_data = self.input_schema.model_validate(input_data).model_dump()
            return True, "succfully validated"
        except Exception as e:
            error_report = str(e)

            error_report = re.sub(
                r"^\s*For further information visit.*$",
                "",
                error_report,
                flags=re.MULTILINE
            ).strip()

            return False, error_report
        
    
    def run(self):
        return f"Succfully Inputting text into element at {self.x_path} with {self.input_data}"


In [141]:
step_1 = InputStep(
    x_path="//input[@id='username']", 
    desc="username",
    input_schema_def={
        "username": {
            "type": "string",
            "min_length": 3
        }
    }
)

step_2 = InputStep(
    x_path="//input[@id='password']", 
    desc="password",
    input_schema_def={
        "password": {
            "type": "string",
            "min_length": 8
        }
    }
)

step_3 = LeftClickStep(
    x_path="//input[@id='sign_in']"
)

In [142]:
from typing import Union, Annotated
from pydantic import Field, TypeAdapter

StepUnion = Annotated[
    Union[LeftClickStep, InputStep],
    Field(discriminator="step_type")
]

In [143]:
class Flow(BaseModel):
    steps: List[StepUnion] = []
    current_step_i: int = 0
    desc: str
    
    @computed_field
    @property
    def current_step(self) -> Step:
        return self.steps[self.current_step_i]

    def progress(self):
        logs = ""
        for i in range(self.current_step_i, len(self.steps)):
            current_step = self.steps[self.current_step_i]
            if hasattr(current_step, "input_schema"):
                if current_step.input_data == None:
                    return "".join([
                        logs,
                        "current step desciption : \n",
                        current_step.desc + " \n",
                        "needs parameters to run \n",
                        str(current_step.input_schema.model_json_schema())
                    ])
            result = current_step.run()
            logs += f"{result} \n"

            if not self.current_step_i == len(self.steps)-1: # we not reached the last step
                self.current_step_i += 1
        
        return f"{logs} \nFlow compeleted"

                
    def send_params_for_current_step(self, data):
        return self.current_step.validate_input(data)
    
    def select_or_change_step(self, idx):
        self.current_step_i = idx
        

In [144]:
login_flow = Flow(
    desc="login the user to the website",
    steps= [
        step_1,
        step_2,
        step_3
    ]
)

In [145]:
class Website(BaseModel):
    flows: List[Flow] = []
    current_flow_i: int = 0

    @computed_field
    @property
    def current_flow(self) -> Flow:
        return self.flows[self.current_flow_i]

    @computed_field
    @property
    def flows_desciption(self) -> str:
        desc = ""
        for i, flow in enumerate(self.flows):
            desc += f"flow {i} => {flow.desc}"

        return desc
    
    def select_or_change_flow(self, idx):
        self.current_flow_i = idx

    def progress(self):
        return self.current_flow.progress()
    
    def send_params_for_current_flow(self, data: dict):
        return self.current_flow.send_params_for_current_step(data)

In [146]:
alcamp = Website(
    flows=[login_flow]
)

In [157]:
json_d = alcamp.model_dump_json()
with open("data/maps/alcamp.json", "w") as f:
    f.write(json_d)

In [147]:
user_flow = alcamp.model_copy(deep=True)

In [ ]:
json_d = user_flow.model_dump_json()
# user_flow = Website.model_validate_json(json_d)

In [149]:
print(user_flow.progress())

current step desciption : 
username 
needs parameters to run 
{'properties': {'username': {'minLength': 3, 'title': 'Username', 'type': 'string'}}, 'required': ['username'], 'title': 'DynamicInputSchema', 'type': 'object'}


In [150]:
user_flow.send_params_for_current_flow({
    "username":"mohamed"
})

(True, 'succfully validated')

In [151]:
print(user_flow.progress())

Succfully Inputting text into element at //input[@id='username'] with {'username': 'mohamed'} 
current step desciption : 
password 
needs parameters to run 
{'properties': {'password': {'minLength': 8, 'title': 'Password', 'type': 'string'}}, 'required': ['password'], 'title': 'DynamicInputSchema', 'type': 'object'}


In [152]:
user_flow.send_params_for_current_flow({
    "password":"mohamed1312"
})

(True, 'succfully validated')

In [153]:
print(user_flow.progress())

Succfully Inputting text into element at //input[@id='password'] with {'password': 'mohamed1312'} 
Left clicking on element at //input[@id='sign_in'] 
 
Flow compeleted


In [154]:
user_flow.flows[0].steps

[InputStep(step_type='input', x_path="//input[@id='username']", desc='username', input_schema_def={'username': {'type': 'string', 'min_length': 3}}, input_data={'username': 'mohamed'}),
 InputStep(step_type='input', x_path="//input[@id='password']", desc='password', input_schema_def={'password': {'type': 'string', 'min_length': 8}}, input_data={'password': 'mohamed1312'}),
 LeftClickStep(step_type='left_click', x_path="//input[@id='sign_in']", desc='Left click on the element')]

In [155]:
json_d = user_flow.model_dump_json()
user_flow = Website.model_validate_json(json_d)

In [156]:
user_flow.flows[0].steps

[InputStep(step_type='input', x_path="//input[@id='username']", desc='username', input_schema_def={'username': {'type': 'string', 'min_length': 3}}, input_data={'username': 'mohamed'}),
 InputStep(step_type='input', x_path="//input[@id='password']", desc='password', input_schema_def={'password': {'type': 'string', 'min_length': 8}}, input_data={'password': 'mohamed1312'}),
 LeftClickStep(step_type='left_click', x_path="//input[@id='sign_in']", desc='Left click on the element')]